In [1]:
import autogluon as ag

In [2]:
@ag.autogluon_method
def myfaketest(args, reporter):
    print(vars(args))
    reporter(accuracy=args.value)

In [3]:
import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

cs = CS.ConfigurationSpace()
value = CSH.UniformFloatHyperparameter('value', lower=0, upper=100)
cs.add_hyperparameter(value)

searcher1 = ag.searcher.RandomSampling(cs)
searcher2 = ag.searcher.RandomSampling(cs)


In [4]:
searcher1

RandomSampling(
ConfigSpace: Configuration space object:
  Hyperparameters:
    value, Type: UniformFloat, Range: [0.0, 100.0], Default: 50.0
.
Number of Trials: 0.
Best Config: {}
Best Reward: 0.0)

In [5]:
import argparse
args = argparse.Namespace()
args.value = 0

myscheduler1 = ag.distributed.DistributedFIFOScheduler(myfaketest, args,
                                                       resource={'num_cpus': 2, 'num_gpus': 1},
                                                       searcher=searcher1,
                                                       num_trials=10,
                                                       reward_attr="accuracy")
myscheduler2 = ag.distributed.DistributedFIFOScheduler(myfaketest, args,
                                                       resource={'num_cpus': 2, 'num_gpus': 1},
                                                       searcher=searcher2,
                                                       num_trials=10,
                                                       reward_attr="accuracy")

[ scheduler local ] : distributed.scheduler - INFO - -----------------------------------------------
[ worker local ] : distributed.diskutils - INFO - Found stale lock file and directory '/home/ubuntu/git/AutoGluon/examples/worker-mzwmw_ly', purging
[ scheduler local ] : distributed.scheduler - INFO - Clear task state
[ scheduler local ] : distributed.dashboard.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: pip install jupyter-server-proxy
[ scheduler local ] : distributed.scheduler - INFO -   Scheduler at:   tcp://172.31.23.33:8781
[ scheduler local ] : distributed.scheduler - INFO -   dashboard at:                     :8787
[ scheduler local ] : distributed.scheduler - INFO - Local Directory:    /tmp/scheduler-dva2ezbk
[ scheduler local ] : distributed.scheduler - INFO - -----------------------------------------------
[ scheduler local ] : distributed.scheduler - INFO - Receive client connection: Remote-9d3c99cc-d429-11e9-b226-2d7d8b8e2

In [6]:
myscheduler1

DistributedFIFOScheduler(
DistributedResourceManager{
(Remote: Remote REMOTE_ID: 0, 
	<Remote: scheduler='tcp://172.31.23.33:8781' processes=1 cores=64>, Resource: NodeResourceManager(64 CPUs, 8 GPUs))
})

In [7]:
myscheduler2

DistributedFIFOScheduler(
DistributedResourceManager{
(Remote: Remote REMOTE_ID: 0, 
	<Remote: scheduler='tcp://172.31.23.33:8781' processes=1 cores=64>, Resource: NodeResourceManager(64 CPUs, 8 GPUs))
})

In [8]:
myscheduler1.run()

[ scheduler local ] : distributed.scheduler - INFO - Receive client connection: Client-worker-9f092ba2-d429-11e9-b2c9-795d6f4b59ac
[ scheduler local ] : distributed.core - INFO - Starting established connection
[ worker local] : {'value': 70.80599896663361, 'task_id': 0}
[ worker local] : {'value': 85.33424924654159, 'task_id': 2}
[ worker local] : {'value': 11.776230130384613, 'task_id': 1}
[ worker local] : {'value': 89.17383667487057, 'task_id': 5}


In [9]:
myscheduler1.join_tasks()

[ worker local] : {'value': 90.40165242050453, 'task_id': 3}
[ worker local] : {'value': 25.8987803706377, 'task_id': 6}
[ worker local] : {'value': 55.11623130816776, 'task_id': 7}
[ worker local] : {'value': 58.60364563381175, 'task_id': 4}
[ worker local] : {'value': 71.1934859783368, 'task_id': 8}
[ worker local] : {'value': 45.65400732718562, 'task_id': 9}


In [10]:
myscheduler2.run()
myscheduler2.join_tasks()

[ worker local] : {'value': 42.26707694066803, 'task_id': 11}
[ worker local] : {'value': 27.55015466787575, 'task_id': 10}
[ worker local] : {'value': 23.202324753275192, 'task_id': 12}
[ worker local] : {'value': 70.32969353942566, 'task_id': 13}
[ worker local] : {'value': 91.1766535016539, 'task_id': 14}
[ worker local] : {'value': 40.056612221382714, 'task_id': 15}
[ worker local] : {'value': 78.23934442548637, 'task_id': 16}
[ worker local] : {'value': 81.00888631782374, 'task_id': 17}
[ worker local] : {'value': 43.53362896443377, 'task_id': 18}
[ worker local] : {'value': 61.382607446031344, 'task_id': 19}


In [11]:
myscheduler1.searcher.get_best_reward()

90.40165242050453

In [12]:
myscheduler2.searcher.get_best_reward()

91.1766535016539